#Hackathon TRACTIAN - SECOMP
Equipe Triple AI

In [1]:
# Instalações necessárias
!pip install openai==0.28
!pip install pypdf
from pypdf import PdfReader
import openai
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.6 MB/s eta 0:00:00


In [2]:
# Configurar a chave da API do ChatGPT
openai.api_key = 'CHAVE API'

In [55]:
# Função simulada para interpretar o comando do usuário
def main_chatbot(user_input):
    # Analisa o input para determinar as funções necessárias
    response = []
    if "transcrição de áudio" in user_input:
        response.append("transcrição de áudio")
    if "manual de instruções" in user_input:
        response.append("manual de instruções")
    if "assistente técnico" in user_input or "descrição de instruções" in user_input or "ferramentas necessárias" in user_input:
        response.append("assistente técnico")
    if "disponibilidade de ferramentas" in user_input:
        response.append("disponibilidade de ferramentas")
    if "ordem de serviço" in user_input:
        response.append("ordem de serviço")
    if "escala de turnos" in user_input:
        response.append("escala de turnos")
    if "relatório de paradas" in user_input:
        response.append("relatório de paradas")
    return response

# Função principal que roteia as requisições
def route_request(user_input, file_path=None, pdf_path=None, tool_catalog=None, manual_text=None, schedule_data=None, employee_data=None, holiday_data=None):
    # Usa o main_chatbot para interpretar a solicitação do usuário
    main_response = main_chatbot(user_input)

    # Armazena as funções a serem chamadas
    results = []  # Lista para coletar resultados de cada função executada

    # Executa a transcrição de áudio se solicitado
    if "transcrição de áudio" in main_response and file_path:
        transcribed_text = transcribe_audio(file_path)
        results.append(f"Transcrição de Áudio: {transcribed_text}")

    # Executa a extração de texto do PDF se solicitado
    if "manual de instruções" in main_response and pdf_path:
        manual_text = extract_pdf_text(pdf_path)
        results.append(f"Texto do Manual: {manual_text}")

    # Executa a função de assistente técnico se solicitado
    if "assistente técnico" or "descrição de instruções" or "ferramentas necessárias" in main_response:
        technician_instructions = help_technician(user_input, tool_catalog, manual_text)
        results.append(f"Instruções para o Técnico: {technician_instructions}")

    # Verifica a disponibilidade de ferramentas se solicitado
    if "disponibilidade de ferramentas" in main_response and schedule_data:
        tool_availability = check_reservations(user_input, schedule_data)
        results.append(f"Disponibilidade de Ferramenta: {tool_availability}")

    # Cria uma ordem de serviço se solicitado
    if "ordem de serviço" in main_response:
        service_order = create_order_of_service(user_input)
        results.append(f"Ordem de Serviço: {service_order}")

    # Otimiza a escala de turnos se solicitado
    if "escala de turnos" in main_response and employee_data and holiday_data:
        schedule = optimize_schedule(employee_data, holiday_data)
        results.append(f"Escala de Turnos: {schedule}")

    # Otimiza o relatório de paradas se solicitado
    if "relatório de paradas" in main_response:
        downtime_analysis = optimize_downtime_report(user_input)
        results.append(f"Relatório de Paradas: {downtime_analysis}")

    # Consolida todos os resultados em uma resposta final
    if results:
        final_response = "\n\n".join(results)
    else:
        final_response = "Não foi possível identificar a solicitação. Por favor, forneça mais detalhes."

    return final_response


In [18]:
def transcribe_audio(file_path):
    with open(file_path, "rb") as audio_file:
        transcript = openai.Audio.transcribe("whisper-1", audio_file)
    return transcript['text']

In [31]:
def extract_pdf_text(pdf_path):
    with open(pdf_path, "rb") as pdf_file:
        reader = PdfReader(pdf_file)
        text = " "
        for i in range(len(reader.pages)):
            page = reader.pages[i]
            text += page.extract_text()
    return text

In [38]:
def help_technician(transcribed_text, tool_catalog, manual_text):
    prompt = f"""
    Você é um assistente técnico para a equipe de manutenção industrial. O chefe enviou o seguinte áudio de instruções que foi transcrito:
    '{transcribed_text}'.

    1. Organize as instruções por escrito e em ordem para o técnico seguir de maneira clara.
    2. Sugira as ferramentas necessárias para cada tarefa das instruções, com base no catálogo:
       {tool_catalog}.
    3. Indique o manual relevante, caso tenha sido mencionado alguma máquina nas intruções,
    e forneça instruções de uso com base no texto do manual e as traduza para o português: {manual_text}.

    Responda como se você fosse o assistente respondendo diretamente ao técnico.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

In [5]:
def check_reservations(tool_id, schedule_data):
    prompt = f"""
    Você é um assistente do almoxarifado. Um funcionário solicitou a ferramenta de ID '{tool_id}'.
    Verifique se esta ferramenta está disponível, consultando a planilha de agendamentos a seguir:
    {schedule_data}.

    Responda 'Disponível' se a ferramenta estiver disponível para uso, ou informe o horário de reserva se já estiver agendada.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

In [6]:
def create_order_of_service(task_description, safety_norms="NR-12"):
    prompt = f"""
    Você é um assistente do planejador de ordens de serviço. Gere uma ordem de serviço completa para a tarefa '{task_description}',
    certificando-se de que todas as instruções necessárias estão incluídas. Siga as normas de segurança aplicáveis, incluindo {safety_norms}.

    Estruture a resposta como uma ordem de serviço detalhada, com um checklist de procedimmentos de segurança e EPI necessários,
    bem como os passos para execução da tarefa, caso aplicável. Lembre-se que o idioma da resposta é português. Tome como base a seguinte estrutura:

    Ordem de Serviço: #977
    Data de Vencimento: 02/11/2024
    Título: Inspeção Sensitiva Motor 150CV
    Status: Em Aberto
    Responsáveis: OPERAÇÃO.
    Categoria: GR - GERAL
    Tags: Sensitiva

    Ativo:
    Nome: TMD BRASIL 2024 / 1. FACTORY / 102 Motor Elétrico Trifásico
    Modelo: Motor Trifásico
    Tags: N/A

    Atividades
    Título: Realizar a inspeção sensitiva - 010
    Tipo de Atividade: #7 IE: INSPEÇÃO DE EQUIPAMENTOS
    Status: Não Iniciada
    Valores estimados de:
    Horas: 10min Executante: 1
    Data de Início Programada
    -
    Data de Fim Programada
    -
    Executante
    -

    Procedimentos
    Título: Procedimento de Inspeção Sensitiva para Compressores de Alta
    Descrição: Este procedimento descreve as etapas para realizar uma inspeção sensitiva em compressores de alta, utilizando os sentidos do técnico para identificar sinais de desgaste, mau funcionamento ou ineficiência.
    Atividade: Realizar a inspeção sensitiva - 010

    Preparação
    1. Equipamentos de Segurança e Ferramentas Necessárias:
    Equipamento de Proteção Individual (EPI): luvas, óculos de proteção, protetores auriculares.
    Ferramentas de medição de temperatura (termômetro infravermelho).
    Equipamento de detecção de vibração (acelerômetro).
    Lanterna para inspeção visual.
    Kit de bloqueio e etiquetagem (LOTO).

    Execução
    2. Inspeção Visual
    OK NÃO OK N/A
    3. Inspeção Auditiva
    OK NÃO OK N/A
    4. Inspeção Térmica
    OK NÃO OK N/A
    5. Inspeção Olfativa
    OK NÃO OK N/A

    Conclusão
    6. Verificações Pós-Manutenção
    Certifique-se de que todas as ferramentas e equipamentos foram removidos da área de trabalho.
    Remova o bloqueio e a etiquetagem (LOTO) e restaure a energia do compressor.
    Realize um teste de funcionamento para garantir que o compressor opere sem problemas.
    Documente todas as observações e ações tomadas durante a inspeção.

    7. Assinatura do Técnico Responsável
    Assinado por ____________________ em ___/___________/____

    Custos
    Itens R$ 0,00
    Apontamentos R$ 0,00
    Outros R$ 0,00
    Total: R$ 0,00

    _________________________________________________
    Assinatura
    _______ /__________________ /___________
    Data

    """
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

In [7]:
def optimize_schedule(employee_data, holiday_data):

    prompt = f"""
    Você é assistente do planejador da empresa encarregado de programar quem vai ficar em cada turno,
    inclusive finais de semana e feriado, tomando cuidado para não deixar sempre a mesma pessoa nos turnos indesejados.
    Crie uma escala otimizada considerando os funcionários disponíveis e os feriados: {employee_data}, {holiday_data}.
    Além disso, a carga horária máxima de cada trabalhador é de 40h num regime 6x1 (um dia off)"""

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message['content']

In [8]:
def optimize_downtime_report(report_text):
    prompt = f"""
    Você é um assistente do planejador encarregado de analisar relatórios de parada para reduzir o tempo de inatividade das máquinas.
    Leia o seguinte relatório de parada e sugira soluções práticas e eficazes para minimizar o tempo parado no futuro:

    Relatório: '{report_text}'

    Forneça uma análise com sugestões de melhoria, considerando manutenção preventiva e possíveis ajustes de processo.
    """
    response = openai.ChatCompletion.create(
        model="gpt-4-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response['choices'][0]['message']['content']

##Exemplos de uso

In [53]:
#arquivos da pasta de exemplo do hackathon que foram upados no colab
tool_catalog = pd.read_csv('catalog.csv')
file_path = "/content/WhatsApp Ptt 2024-10-21 at 19.29.47.ogg"
pdf_path = "/content/WEG-w22-three-phase-electric-motor-50029265-brochure-english-web.pdf"

In [39]:
# Transcreve o áudio
transcribed_text = transcribe_audio(file_path)
# Extrai o texto do PDF
manual_text = extract_pdf_text(manual_instructions)
# print(help_technician(transcribed_text, tool_catalog, manual_text))

Olá! Aqui estão as instruções organizadas e claras para as tarefas que precisam ser realizadas no domingo:

1. **Lubrificação da Linha 3:**
   - **Equipamento:** Rolamentos na linha 3.
   - **Ação:** Aplicar o lubrificante de código azul 6624.
   - **Ferramentas Necessárias:** Chave de fenda ou Allen (conforme necessário para abrir o compartimento do rolamento).
   - **Detalhes Adicionais:** Verificar a ficha técnica do equipamento para a quantia exata de lubrificante a ser utilizada.

2. **Verificação de Óleo na Linha 12:**
   - **Equipamento:** Desencapadora da linha 12.
   - **Ação:** Checar os níveis de óleo.
   - **Ferramentas Necessárias:** Ferramenta para medição de nível de óleo.
   - **Detalhes Adicionais:** Observar possíveis picos de temperatura e se estão correlacionados com o nível de óleo.

3. **Substituição de Filtro de Ar:**
   - **Equipamento:** Compressor 5 na central.
   - **Ação:** Trocar o filtro de ar já que está crítico.
   - **Ferramentas Necessárias:** Chave de

In [40]:
transcribed_text = "O chefe me disse para verificar a pressão no tanque A, checar o óleo e substituir o filtro. Também pediu para verificar o motor elétrico trifásico"
print(help_technician(transcribed_text, tool_catalog, manual_text))

Olá! Aqui estão as instruções organizadas e detalhadas para a manutenção que o chefe solicitou. Segue também uma lista das ferramentas necessárias baseadas no nosso catálogo.

### Instruções de Manutenção:

1. **Verificar a pressão no tanque A:**
   - Utilize o manômetro adequado. Certifique-se de que a pressão está dentro dos parâmetros operacionais recomendados.
   - Ferramentas sugeridas: Não específicas no catálogo fornecido.

2. **Checar o óleo:**
   - Verifique o nível e a qualidade do óleo. Substitua se estiver abaixo do nível ou se notar impurezas.
   - Ferramentas sugeridas: Não específicas no catálogo fornecido.

3. **Substituir o filtro:**
   - Desligue a máquina e remova o filtro usado. Substitua por um novo e assegure-se de que está devidamente instalado.
   - Ferramentas sugeridas: Kit de chaves (Código SAP: MAT905) pode ser necessário dependendo do filtro.

4. **Verificar o motor elétrico trifásico:**
   - Inspeção visual para verificar sinais de desgaste ou danos nos ca

In [41]:
task_description = "Inspeção de segurança na prensa hidráulica"
safety_norms = "NR-12"
print(create_order_of_service(task_description, safety_norms))

Ordem de Serviço: #977  
Data de Vencimento: 02/11/2024  
Título: Inspeção de segurança na prensa hidráulica  
Status: Em Aberto  
Responsáveis: OPERAÇÃO  
Categoria: GR - GERAL  
Tags: Segurança, NR-12

Ativo:  
Nome: TMD BRASIL 2024 / 1. FACTORY / Prensa Hidráulica  
Modelo: Prensa Hidráulica  
Tags: N/A

Atividades  
Título: Realizar a inspeção de segurança - 010  
Tipo de Atividade: #7 IE: INSPEÇÃO DE EQUIPAMENTOS  
Status: Não Iniciada  
Valores estimados de:  
Horas: 30 min  
Executante: 1  
Data de Início Programada: -  
Data de Fim Programada: -  
Executante: -

Procedimentos  
Título: Procedimento de Inspeção de Segurança para Prensa Hidráulica  
Descrição: Este procedimento descreve as etapas para realizar uma inspeção de segurança em prensas hidráulicas, focando na identificação de riscos mecânicos e elétricos de acordo com a NR-12.

Preparação  
1. Equipamentos de Segurança e Ferramentas Necessárias:  
Equipamento de Proteção Individual (EPI): Capacete, luvas de proteção me

In [42]:
report_text = "A máquina X apresentou superaquecimento durante a produção. O processo foi interrompido por 2 horas para resfriamento e ajuste técnico."
print(optimize_downtime_report(report_text))

A partir do relatório fornecido sobre o superaquecimento da máquina X, podemos identificar que um dos principais problemas enfrentados é a incapacidade de manter a máquina operando em temperaturas seguras, ocasionando paradas necessárias para o resfriamento e ajuste técnico. Para minimizar o tempo de parada no futuro e melhorar a eficiência geral da produção, sugiro as seguintes medidas:

1. **Implementação de Manutenção Preventiva Regular:**
   - **Inspeções térmicas periódicas**: Utilizar termômetros infravermelhos ou câmeras termográficas para detectar áreas de superaquecimento antes que elas causem paradas. Estas inspeções devem ser agendadas com base na carga de trabalho e nos ciclos operacionais da máquina.
   - **Verificação e limpeza de componentes**: Assegurar que todos os componentes, especialmente os que estão diretamente relacionados com o resfriamento da máquina, sejam limpos e verificados regularmente para evitar acúmulo de sujeira e gunk que possam afetar o desempenho té

In [46]:
user_input = "O chefe me disse para verificar a pressão no tanque A, checar o óleo e substituir o filtro. Também pediu para verificar o motor elétrico trifásico"
print(route_request(user_input))

Olá! Aqui estão as instruções organizadas e detalhadas para facilitar a execução das tarefas:

1. **Verificar a pressão no tanque A**
   - **Ferramentas necessárias:** 
     - Manômetro apropriado para verificar a pressão.
     - Luvas de segurança e óculos de proteção.

2. **Checar o óleo**
   - **Ferramentas necessárias:**
     - Vareta de medição para verificar o nível de óleo.
     - Luvas de segurança.

3. **Substituir o filtro**
   - **Ferramentas necessárias:**
     - Chave apropriada para remover e instalar o filtro.
     - Recipiente para descarte do óleo do filtro antigo.
     - Luvas de segurança e óculos de proteção.

4. **Verificar o motor elétrico trifásico**
   - **Ferramentas necessárias:**
     - Multímetro para verificar a tensão e a continuidade elétrica.
     - Chave de fenda para abrir o painel do motor, se necessário.
     - Luvas isolantes e óculos de proteção.

Cada tarefa deve ser realizada com atenção e seguindo todas as normas de segurança. No caso do motor e

In [48]:
print(route_request(transcribed_text, tool_catalog, manual_text))

Olá! Abaixo estão organizadas as instruções de manutenção para você seguir neste domingo:

1. **Lubrificação dos rolamentos na linha 3:**
   - Utilize o lubrificante código azul 6624 já disponível no estoque.
   - Consulte a ficha técnica do equipamento para verificar a quantidade correta de lubrificante.

2. **Verificação do nível de óleo na desencapadora da linha 12:**
   - Verifique as medições e assegure-se de que o nível de óleo está correto para evitar picos de temperatura e possíveis paradas do equipamento.

3. **Substituição do filtro de ar no compressor 5:**
   - Localize o filtro novo sob a bancada no almoxarifado.
   - Proceda com a substituição do filtro de ar que já deveria ter sido trocada na última parada.

4. **Inspeção na bomba de circulação no canto direito junto ao compressor:**
   - Verifique a bomba de circulação quanto a qualquer ruído estranho e inspecione seu funcionamento geral.

### Ferramentas Necessárias:
- Para a lubrificação dos rolamentos:
  - Usar uma pi

In [50]:
print(route_request(file_path, tool_catalog, manual_text))

Olá,

Como não foi fornecido o texto exato das instruções do áudio, vou criar um exemplo genérico de como você poderia organizar as instruções de manutenção industrial com base em práticas comuns de manutenção. Assim que você tiver as instruções específicas, você pode seguir este modelo para organizá-las e identificar as ferramentas necessárias.

### Instruções de Manutenção:

1. **Inspeção Visual e Limpeza:**
   - **Descrição da Tarefa:** Verifique visivelmente a máquina por quaisquer sinais de desgaste, danos ou acúmulo de detritos. Limpe a máquina para remover poeira, sujeira e outros contaminantes.
   - **Ferramentas Necessárias:** Pano de limpeza, escova de cerdas macias, aspirador de pó para equipamentos industriais.

2. **Lubrificação:**
   - **Descrição da Tarefa:** Aplique lubrificante nos componentes móveis para garantir que eles funcionem suavemente sem excesso de atrito ou superaquecimento.
   - **Ferramentas Necessárias:** Óleo lubrificante ou graxa, pistola de graxa, pano

In [56]:
print(route_request("Preciso transcrever um áudio e fornecer instruções para o técnico.", file_path, pdf_path, tool_catalog))

Instruções para o Técnico: Olá, aqui estão as instruções organizadas e claras para você seguir:

1. **Verificar o Equipamento**:
   - Inspecione o equipamento para identificar sinais visíveis de desgaste ou danos.
   - Ferramentas sugeridas: Torquímetro 40-200Nm (Código SAP: MAT904).

2. **Substituição de Componentes Desgastados**:
   - Substitua quaisquer peças que mostrem sinais de desgaste evidente, como discos de corte ou serras.
   - Ferramentas sugeridas: Serra Circular (Código SAP: MAT001), Disco de Corte (Código SAP: MAT002).

3. **Ajuste e Calibração**:
   - Faça os ajustes necessários para garantir que o equipamento esteja operando na eficiência máxima.
   - Ferramentas sugeridas: Conjunto de Chaves Allen (Código SAP: MAT905), Chave Estrela 12mm (Código SAP: MAT906).

4. **Teste Funcional**:
   - Após a manutenção, realize testes para assegurar que o equipamento está funcionando corretamente.
   - Ferramenta sugerida: Martelo de Borracha (Código SAP: MAT903) para testes leves